In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
import numpy as np
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
load_dotenv()

True

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
embedding = OpenAIEmbeddings(model = 'text-embedding-3-small')
embedding

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7cf7c8ae8a60>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7cf7b572c550>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [4]:
text = "This is my first tutorial of OpenAI embedding"
embedded_sentence = embedding.embed_query(text)
embedded_sentence

[-0.006263286806643009,
 -0.013730396516621113,
 0.031753670424222946,
 -0.019931219518184662,
 -0.0026347823441028595,
 -0.026007119566202164,
 0.005329187959432602,
 0.014196026138961315,
 -0.011890591122210026,
 0.014832007698714733,
 0.055830132216215134,
 -0.05523957684636116,
 -0.009664654731750488,
 -0.0576472207903862,
 0.01585412211716175,
 -0.006410925183445215,
 -0.00679137883707881,
 -0.014479946345090866,
 0.0003557878953870386,
 0.027892351150512695,
 0.022531932219862938,
 -0.021657457575201988,
 -0.03600111976265907,
 0.05024257302284241,
 0.015547487884759903,
 -0.0611451230943203,
 0.057147521525621414,
 0.04960659146308899,
 -0.011970088817179203,
 -0.03082241117954254,
 0.012889991514384747,
 -0.014661655761301517,
 -0.01857975870370865,
 -0.030413566157221794,
 -0.01495693251490593,
 0.01736457832157612,
 0.007279722020030022,
 0.022497862577438354,
 -0.03293478116393089,
 0.030617987737059593,
 0.03223065659403801,
 -0.019851721823215485,
 -0.013775823637843132,
 

In [5]:
np.array(embedded_sentence).shape

(1536,)

In [6]:
embedding_512 = OpenAIEmbeddings(model = 'text-embedding-3-small', dimensions = 512)
embedded_sentence_512 = embedding_512.embed_query(text)
np.array(embedded_sentence_512).shape

(512,)

In [7]:
loader = TextLoader('speech.txt')
doc = loader.load()
doc

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness be

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 512, chunk_overlap = 64)
chunked_doc = text_splitter.split_documents(doc)
chunked_doc

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_co

#### Vector Embedding and Vector StoreDB

In [9]:
db = Chroma.from_documents(chunked_doc,embedding)
db

In [10]:
query = "We have borne with their present government through all these bitter months because of that friendship—exercising"
search_results = db.similarity_search(query)
print(search_results)

[Document(metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward all who are in fact loyal to their neighbors and to the government in'), Document(metadata={'source': 'speech.txt'}, page_content='It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is